**NOTE** This notebook is work under progress

# Interactive exploration of current errors in pandas docstrings

*DISCLAIMER: This notebook is based on the one uploaded by @dujm [here](https://github.com/python-sprints/pandas-mentoring/blob/master/notebooks/docstring_error_interactive.ipynb)*


This notebook will help you detect which errors are still present on some of the docstrings of pandas, so that you can select one of them, fix it, and submit a PR to the [pandas repository](https://github.com/pandas-dev/pandas). 

**IMPORTANT!** Before starting to work on fixing an error, check that nobody is already working on it by searching the issues and PRs in the pandas repository. If you nobody is doing so, open an issue and let others know you will be fixing that docstring.

This script currently supports pandas version >= 0.25.0

Let's start by importing the necessary packages:

In [2]:
import os

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import ipywidgets as widgets
import qgrid

## *Static exploration*

## 1. Generate a .json containing all current errors

This step was automatically done if you are running this notebook from Binder. Keep in mind that the .json file is updated every 15 minutes, so it might be outdated. When you select an error to work on, double check that nobody has submitted an issue to work on it already.

If you want to generate the .json file locally, simply run the following command from your pandas clone:

`./scripts/validate_docstrings.py --format=json > /path/to/json/pandas_docstring_errors.json`

## 2. Plot a table describing the errors

We will plot a table that indicates which pandas functions still have an error in their docstrings. The specific error code and description will be also described.

In [4]:
file = 'pandas_docstring_errors.json'
df = (pd.read_json(file)
            .transpose()
            .filter(items=['errors', 'file', 'file_line'])
            .explode('errors')
            .dropna()
            .reset_index()
            .rename(columns={"index": "function"})
     )
df[['error_code','error_description']] = pd.DataFrame(df.errors.tolist())
df = df.drop(["errors"], axis=1)

df

,function,file,file_line,error_code,error_description
0,pandas.Categorical,pandas/core/arrays/categorical.py,209,PR01,Parameters {fastpath} not documented
1,pandas.Categorical.__array__,pandas/core/arrays/categorical.py,1267,PR01,Parameters {dtype} not documented
2,pandas.Categorical.categories,pandas/core/arrays/categorical.py,396,SA04,"Missing description for See Also ""rename_categ..."
3,pandas.Categorical.categories,pandas/core/arrays/categorical.py,396,SA04,"Missing description for See Also ""reorder_cate..."
4,pandas.Categorical.categories,pandas/core/arrays/categorical.py,396,SA04,"Missing description for See Also ""add_categori..."
...,...,...,...,...,...
2313,pandas.wide_to_long,pandas/core/reshape/melt.py,195,EX03,flake8 error: C403 Unnecessary list comprehens...
2314,pandas.wide_to_long,pandas/core/reshape/melt.py,195,EX03,flake8 error: E124 closing bracket does not ma...
2315,pandas.wide_to_long,pandas/core/reshape/melt.py,195,EX03,flake8 error: E261 at least two spaces before ...
2316,pandas.wide_to_long,pandas/core/reshape/melt.py,195,EX03,flake8 error: E741 ambiguous variable name 'l'...


## 3. Count number of functions with errors per error type

In [5]:
df_code = df['error_code'].value_counts().reset_index()
df_code.columns = ['error_code','counts']

df_code

,error_code,counts
0,GL08,407
1,RT03,328
2,SA04,313
3,PR01,253
4,PR07,249
5,EX02,174
6,EX03,166
7,RT02,135
8,SS06,108
9,PR02,101


## 4. Count number of errors per function

In [6]:
df_function = df['function'].value_counts().reset_index()
df_function.columns = ['function','counts']

df_function

,function,counts
0,pandas.HDFStore.append,14
1,pandas.core.groupby.DataFrameGroupBy.boxplot,13
2,pandas.PeriodIndex,13
3,pandas.merge_asof,11
4,pandas.Series.cat.remove_unused_categories,9
...,...,...
1185,pandas.tseries.offsets.Day.is_anchored,1
1186,pandas.DataFrame.dropna,1
1187,pandas.tseries.offsets.WeekOfMonth.onOffset,1
1188,pandas.ExcelFile.parse,1


## *Interactive exploration* 

Select an error from the following dropdown menu to see a complete description and example of it, and the number of that type of errors in pandas:

In [115]:
# Create dropdown widget
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    return unique

dropdown_widget = widgets.Dropdown(options=unique_sorted_values(df_code.error_code),
                                   description='Error:')
dropdown_widget

## TODO: Use observe to print error description

Dropdown(description='Error:', options=('EX02', 'EX03', 'GL01', 'GL02', 'GL08', 'PR01', 'PR02', 'PR06', 'PR07'…

You can filter the following table by the error code you want to work on, or its function.

In [112]:
# Create qgrid widget
qgrid_widget = qgrid.show_grid(df, grid_options={'forceFitColumns': True})
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…